# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - ответить на следующие вопросы:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Общая информация о данных хранится в таблице, расположенной по адресу `/datasets/data.csv`. Перед ответом на поставленные вопросы необходим обзор данных.

Далее, в ходе предобработки данных, нужно определить и обработать пропущенные значения, при необходимости заменить тип данных, определить и удалить дубликаты, выделить леммы в значениях столбца с целями получения кредита, а также категоризировать данные.

По результатам исследования необходимо сделать общий вывод.

Таким образом, исследование будет состоять из четырех шагов:
1. Обзор данных
2. Предобработка данных
3. Ответ на поставленные вопросы
4. Формулировка вывода

**Описание данных**
- _children_ — количество детей в семье
- _days_employed_ — общий трудовой стаж в днях
- _dob_years_ — возраст клиента в годах
- _education_ — уровень образования клиента
- _education_id_ — идентификатор уровня образования
- _family_status_ — семейное положение
- _family_status_id_ — идентификатор семейного положения
- _gender_ — пол клиента
- _income_type_ — тип занятости
- _debt_ — имел ли задолженность по возврату кредитов
- _total_income_ — ежемесячный доход
- _purpose — цель_ получения кредита

## Шаг 1. Откройте файл с данными и изучите общую информацию

Открытие файла и знакомство с данными:

In [1]:
import pandas as pd # импорт библиотеки pandas
df = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение его в переменной df
df.head(10) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получение общей информации о данных:

In [2]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Таблица состоит из 12 столбцов, содержащих общую информацию о заемщике.
Обнаружены следующие проблемы:
1. В столбцах, содержащих данные об общем трудовом стаже и ежемесячном доходе присутствуют пропуски
2. Общий трудовой стаж и ежемесячный доход указаны типом данных _float64_
3. В столбце с возрастом клиента присутствуют нулевые значения
4. Данные об образовании заемщика и поле клиента указаны в разном регистре
5. Слова в данных о цели получения кредита записаны в разных формах

Таким образом, перед ответом на поставленные в исследовании вопросы необходимо провести предобработку существующих данных

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
display(df.isna().sum()) # подсчет общего числа пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах с данными об общем трудовом стаже и ежемесячном доходе более 10% пропусков. Если пропуски в данных об общем трудовом стаже никак не повлияют на ответы на вопросы, поставленные в задаче исследования, то данные о ежемесячном доходе нам необходимы, следовательно для обработки пропусков будет использоваться метод _fillna()_.


Прежде, чем приступить к заполнению пропусков, необходимо заменить отрицательные значения на абсолютные:

In [4]:
for days in df['days_employed']: # цикл для перевода всех отрицательных значений столбца 'days_employed' в абсолютные
    if days < 0:
        df['days_employed'] = df['days_employed'].abs()
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Можно заметить, что в значениях трудового стажа встречаются слишком большие значения, например 363000, врядли у человека может быть стаж около 1000 лет. Предположительно, данные значения заполнены не днями, а часами, поэтому для корректной обработки необходимо перевести данные значений в дни, разделив их на 24:

In [5]:
df.loc[df['days_employed'] >= 36500, 'days_employed'] = df['days_employed'] / 24 # приведение аномальных значений столбца 'days_employed' к нормальным
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


После приведения значений в порядок необходимо заполнить пропуски средним арфиметическим значением для каждого типа занятости:

In [6]:
# замена пропущенных значений столбца 'days_employed' на среднее арифметическое для каждого типа занятости
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('mean'))

Пропущенные значения столбца с информацией о ежемесячном доходе заменять на среднее арифметическое будет некорректно, так как разница в значениях слишком высокая, их необходимо заменить на медианное значение:

In [7]:
# замена пропущенных значений столбца 'total_income' на медианное для каждого типа занятости
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

В столбце с информацией о возрасте заемщика замечены нулевые значения. Данное значение является странным, вероятнее всего разработчики таким образом передают в данных пропущенное значение возраста. Проверим количество нулевых значений в возрасте заемщика:

In [8]:
zero_dob_years = len(df[df['dob_years'] == 0]) # определение количества нулевых значений столбца 'dob_years'
all_dob_years = len(df['dob_years']) # определение общего количества значений столбца 'dob_years'
percent_zero_dob_years = zero_dob_years/all_dob_years*100 # определение процентного соотношения нулевых значений к общему числу
print(f'Доля нулевых значений возраста заемщика: {percent_zero_dob_years}%') # вывод результата

Доля нулевых значений возраста заемщика: 0.4692218350754936%


Доля нулевых значений возраста заемщика от общего числа данных менее 1%, следовательно на дальнейшее исследование они не повлияют и их можно оставить без изменений.

Повторно убедимся, что все пропуски удалены:

In [9]:
display(df.isna().sum()) # вывод на экран количества пропусков после обработки

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Пропущенные значения обработаны, пропуски отсутствуют. Использовался метод _fillna()_, так как удаление 10% строк могло сильно повлиять на результаты исследования и ответы на поставленные вопросы.

Пропуски в значениях могли появиться в результате того, что заемщик не указал свои данные, либо в результате сбоя работы системы. 

### Замена типа данных

В столбце с трудовым стажем заемщика указаны данные типа _float64_. Данный тип является некорректным, необходимо изменить тип на _int_, так как количество дней - это целое число, и не может быть дробным. Также, изучив данные об общем трудовом стаже, можно заметить, что трудовой стаж некоторых категорий заемщиков указан отрицательными значениями, для удобства необходимо заменить их на абсолютные методом _abs()_:

In [10]:
df['days_employed'] = df['days_employed'].astype('int') # изменение типа данных столбца 'days_employed'

Для удобства также заменим тип данных у столбца с информацией о ежемесячном доходе заемщика:

In [11]:
df['total_income'] = df['total_income'].astype('int') # изменение типа данных столбца 'days_employed'

Также для дальнейшего удобства работы с данными приведем значения стобцов с инофрмацией об образовании и пола заемщика к единому нижнему регистру:

In [12]:
df['education'] = df['education'].str.lower() # приведение значений столбца 'education' к единому нижнему регистру
df['gender'] = df['gender'].str.lower() # приведение значений столбца 'gender' к единому нижнему регистру

Проверим внесенные изменения:

In [13]:
display(df.head(10)) # получение первых 10 строк таблицы df
df.info() # получение общей информации о данных в таблице df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Для удобства работы с данными были произведены следующие операции:
1. Типы данных столбоцов с информацией о трудовом стаже и ежемесячном доходе заемщика были преобразованы из _float64_ в _int64_ методом _astype_, так как с помощью данного метода мы можем преобразовать любой тип данных именно в тот, который нам необходим
2. Отрицательные значения столбца с тредовым стажем были преобразованы в положительные методом _abs_
3. Значения данных с разными регистрами были приведены к единому нижнему регистру методом _lower()_

### Обработка дубликатов

Произведем поиск явных дубликатов таблицы и выведем их количество:

In [14]:
display(df.duplicated().sum()) # количество явных дубликатов таблицы

71

Удалим явные дубликаты методом _drop_duplicates()_ с удалением старых индексов и созданием новых:

In [15]:
df = df.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов таблицы
display(df.duplicated().sum()) # проверка на наличие явных дубликатов

0

Явные дубликаты удалены. Проверим необходимые для проведения анализа столбцы на наличие неявных дубликатов:

In [16]:
display(df.loc[:,'children'].sort_values().unique()) # вывод уникальных значений столбца 'children' на экран


array([-1,  0,  1,  2,  3,  4,  5, 20])

В столбце с количеством детей заметны артефакты. Вероятнее всего, они образовались в результате сбоя системы, такие данные необходимо обработать:

In [17]:
df['children'] = df['children'].abs() # приведение отрицательных артефактов столбца 'children' к абсолютным значениям
median_children = df['children'].sort_values().median() # определение медианного значения столбца 'children' и сохранение его в переменной median_children
df.loc[df['children'] == 20, 'children'] = median_children.astype('int') # замена некорректного количества детей 20 на медианное значение
display(df.loc[:,'children'].sort_values().unique()) # повторный вывод уникальных значений столбца 'children' на экран для проверки

array([0, 1, 2, 3, 4, 5])

Проверка столбца _family_status_ на наличие неявных дубликатов:

In [18]:
display(df.loc[:,'family_status'].unique()) # вывод уникальных значений столбца 'family_status' на экран

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Проверка столбца _total_income_ на наличие неявных дубликатов:

In [19]:
display(df.loc[:,'total_income'].unique()) # вывод уникальных значений столбца 'total_income' на экран

array([253875, 112080, 145885, ...,  89672, 244093,  82047])

Проверка столбца _purpose_ на наличие неявных дубликатов:

In [20]:
display(df.loc[:,'purpose'].unique()) # вывод уникальных значений столбца 'purpose' на экран

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Проверка столбца _debt_ на наличие неявных дубликатов:

In [21]:
display(df.loc[:,'debt'].unique()) # вывод уникальных значений столбца 'debt' на экран

array([0, 1])

**Вывод**

В ходе проверки на наличие явных дубликатов было выявлено 71 совпадение, указанные дубликаты были удалены из таблицы. Причиной их возникновения, вероятно, был сбой системы.

В ходе проверки на неявные дубликаты, в столбце с количеством детей были обнаружены артефакты, которые были обработаны:
1. Артефакт отрицательного значения был приведен к абсолютному методом _abs()_
2. Атефакт с количеством 20 был заменен на медианное значение столбца

В остальных столбцах, необходимых для ислледования, невяных дубликатов не обнаружено.

### Лемматизация

В столбце с целями получения кредита множество слов написано в разной форме. Для дальнейшей категоризации необходимо применить к данному столбцу лемматизацию:

In [22]:
from pymystem3 import Mystem # импортируем библиотеку pymystem3
m = Mystem() # сохраняем библиотеку в переменной m

def df_lemma(df): # напишем функцию, возвращающую лемматизированное значение
    lemmas = m.lemmatize(df['purpose']) 
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if ('жилье' in lemmas) or ('недвижимость' in lemmas):
        return 'недвижимость'

df['purpose_category'] = df.apply(df_lemma, axis=1) # создаем новый столбец с лемматизированными значениями категорий
display(df.head(10)) # вывод на экран первых 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [23]:
df['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

**Вывод**

В таблицу был добавлен новый столбец с лемматизированными целями получения кредита, благодаря которым будет проще категоризировать данные в дальнейшем.

### Категоризация данных

Необходимо провести категоризацию данных по наличию детей в семье:

In [24]:
def children_group(children): # напишем функцию для разделения заемщиков на категории по количеству детей
    if children == 0:
        return 'нет детей'
    if children == 1:
        return '1 ребенок'
    if children == 2:
        return '2 детей'
    return 'многодетная семья'
df['children_category'] = df['children'].apply(children_group) # добавим новый столбец с категориями по количеству детей в таблицу
display(df.head(10)) # вывод на экран первых 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость,1 ребенок
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль,1 ребенок
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование,многодетная семья
4,0,14177,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,недвижимость,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,недвижимость,нет детей
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,образование,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,свадьба,2 детей
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,недвижимость,нет детей


Необходимо провести категоризацию данных по уровню дохода заемщика:

In [25]:
def total_income_group(total_income): # напишем функцию для разделения заемщиков на категории по уровню дохода
    if total_income <= 36000:
        return 'до 36000'
    if total_income <= 72000:
        return 'от 36000 до 72000'
    if total_income <= 100000:
        return 'от 72000 до 100000'
    if total_income <= 150000:
        return 'от 100000 до 150000'
    if total_income <= 250000:
        return 'от 150000 до 250000'
    if total_income <= 500000:
        return 'свыше 500000'
df['total_income_category'] = df['total_income'].apply(total_income_group) # добавим новый столбец с категориями по уровню дохода в таблицу
display(df.head(10)) # вывод на экран первых 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость,1 ребенок,свыше 500000
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль,1 ребенок,от 100000 до 150000
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость,нет детей,от 100000 до 150000
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование,многодетная семья,свыше 500000
4,0,14177,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,от 150000 до 250000
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,недвижимость,нет детей,свыше 500000
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,недвижимость,нет детей,от 150000 до 250000
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,образование,нет детей,от 100000 до 150000
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,свадьба,2 детей,от 72000 до 100000
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,недвижимость,нет детей,от 100000 до 150000


**Вывод**

Была произведена категоризация данных по наличию детей в семье заемщика, а также по уровню дохода заемщика с помощью написания функции с определенными условиями. Категоризация по семейному положению и целям кредита не требуется, так как она уже была произведена ранее.

Категоризация была выполнена с целью упрощения дальнейшего исследования и ответа на поставленные вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для удобства работы с данными перед ответом на поставленный вопрос создадим сводную таблицу, в которой отразим количество должников и количество добросовестных заемщиков, в чьих семьях есть дети и нет детей:

In [26]:
df_pivot_children_category = df.pivot_table(
    index=['children_category'], 
    columns='debt', values='children', 
    aggfunc='count'
) # создание сводной таблицы со столбцом 'children_category' для группировки данных, а также столбцом 'debt', по значениям которого происходит группировка
display(df_pivot_children_category) # вывод на экран таблицы

debt,0,1
children_category,,
1 ребенок,4410,445
2 детей,1858,194
многодетная семья,349,31
нет детей,13096,1071


Рассчитаем процент должников от общего числа заемщиков для каждой категории:

In [27]:
df_pivot_children_category['debt_persent'] = df.groupby('children_category')['debt'].mean()*100 # создание в таблице столбца с процентом должников по категориям
display(df_pivot_children_category.sort_values(by='debt_persent', ascending=False) \
                                  .style.format({'debt_persent':'{:.2f}%'})) # вывод на экран таблицы отсоритированной по убыванию таблицы

debt,0,1,debt_persent
children_category,,,
2 детей,1858,194,9.45%
1 ребенок,4410,445,9.17%
многодетная семья,349,31,8.16%
нет детей,13096,1071,7.56%


**Вывод**

По полученным данным можно заметить, что процент должников по кредиту в семьях с детьми выше, то есть семьи без детей возвращают кредит в срок чаще, чем семьи с детьми. Также, семьи без детей берут кредит в 2 раза чаще.

Отсюда можно сделать вывод, что вовзврат кредита в срок зависит от наличия в семье детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для удобства работы с данными перед ответом на поставленный вопрос создадим сводную таблицу, в которой отразим количество должников и количество добросовестных заемщиков, в зависимости от семейного положения:

In [28]:
df_pivot_family_status_category = df.pivot_table(
    index=['family_status'], 
    columns='debt', values='family_status_id', 
    aggfunc='count'
) # создание сводной таблицы со столбцом 'family_status' для группировки данных, а также столбцом 'debt', по значениям которого происходит группировка
display(df_pivot_family_status_category) # вывод на экран таблицы

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931


Рассчитаем процент должников от общего числа заемщиков для каждой категории:

In [29]:
df_pivot_family_status_category['debt_persent'] = df.groupby('family_status')['debt'].mean()*100 # создание в таблице столбца с процентом должников по категориям
display(df_pivot_family_status_category.sort_values(by='debt_persent', ascending=False) \
                                       .style.format({'debt_persent':'{:.2f}%'})) # вывод на экран таблицы отсоритированной по убыванию таблицы

debt,0,1,debt_persent
family_status,,,
Не женат / не замужем,2536,274,9.75%
гражданский брак,3763,388,9.35%
женат / замужем,11408,931,7.55%
в разводе,1110,85,7.11%
вдовец / вдова,896,63,6.57%


**Вывод**

По полученным данным видно, что наивысший процент должников среди заемщиков со статусом _"не женат / не замужем"_, а наименьший процент должников у замещиков со статусом _"вдовец / вдова"_, однако это идентичные понятия, так как в обоих случаях человек находится не в браке.

Категория заемщиков со статусом _"женат / замужем"_ находится в среднем положении таблицы, однако, примечательно, что именно эта категория самая многочисленная по количеству заемщиков, что является вполне предсказуемым, ведь семьям часто нужны денежные средства на различные цели.

Еще одна интересная закономерность заключается в том, что заемщики, состоящие или побывавшие в браке, добросовестнее относятся к выполнению своих обязательнств по кредиту.

Исходя из этого, можно утверждать, что между семейным положением и возвратом кредита в срок есть определенная зависимость.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для удобства работы с данными перед ответом на поставленный вопрос создадим сводную таблицу, в которой отразим количество должников и количество добросовестных заемщиков, в зависимости от уровня дохода:

In [30]:
df_pivot_total_income_category = df.pivot_table(
    index=['total_income_category'], 
    columns='debt', values='total_income', 
    aggfunc='count'
) # создание сводной таблицы со столбцом 'total_income_category' для группировки данных, а также столбцом 'debt', по значениям которого происходит группировка
display(df_pivot_total_income_category) # вывод на экран таблицы

debt,0,1
total_income_category,,
до 36000,71,4
от 100000 до 150000,6536,624
от 150000 до 250000,6449,569
от 36000 до 72000,1446,107
от 72000 до 100000,2592,243
свыше 500000,2411,180


Рассчитаем процент должников от общего числа заемщиков для каждой категории:

In [31]:
df_pivot_total_income_category['debt_persent'] = df.groupby('total_income_category')['debt'].mean()*100 # создание в таблице столбца с процентом должников по категориям
display(df_pivot_total_income_category.sort_values(by='debt_persent', ascending=False) \
                                      .style.format({'debt_persent':'{:.2f}%'})) # вывод на экран таблицы отсоритированной по убыванию таблицы

debt,0,1,debt_persent
total_income_category,,,
от 100000 до 150000,6536,624,8.72%
от 72000 до 100000,2592,243,8.57%
от 150000 до 250000,6449,569,8.11%
свыше 500000,2411,180,6.95%
от 36000 до 72000,1446,107,6.89%
до 36000,71,4,5.33%


**Вывод**

По полученным данным можно утверждать, что заемщики с более низким уровнем дохода добросовестнее относятся к своим обязательствам по возврату кредита в срок, однако видно, что данная категория заемщиков гораздо меньше оставшихся.

Исходя из этого, видна неожиданная зависимость между уровнем дохода и возвратом кредита в срок.

Предположительно данная закономерность образована из размера кредита, так как заемщики с более низким уровнем дохода, как правило, берут в долг меньшую сумму на меньший срок, и вероятнее, ответственнее относятся к решению получения займа.

- Как разные цели кредита влияют на его возврат в срок?

Для удобства работы с данными перед ответом на поставленный вопрос создадим сводную таблицу, в которой отразим количество должников и количество добросовестных заемщиков, в зависимости от уровня дохода:

In [32]:
df_pivot_purpose_category = df.pivot_table(
    index=['purpose_category'], 
    columns='debt', values='purpose', 
    aggfunc='count'
) # создание сводной таблицы со столбцом 'purpose_category' для группировки данных, а также столбцом 'debt', по значениям которого происходит группировка
display(df_pivot_purpose_category) # вывод на экран таблицы

debt,0,1
purpose_category,,
автомобиль,3903,403
недвижимость,10029,782
образование,3643,370
свадьба,2138,186


Рассчитаем процент должников от общего числа заемщиков для каждой категории:

In [33]:
df_pivot_purpose_category['debt_persent'] = df.groupby('purpose_category')['debt'].mean()*100 # создание в таблице столбца с процентом должников по категориям
display(df_pivot_purpose_category.sort_values(by='debt_persent', ascending=False) \
                                 .style.format({'debt_persent':'{:.2f}%'})) # вывод на экран таблицы отсоритированной по убыванию таблицы

debt,0,1,debt_persent
purpose_category,,,
автомобиль,3903,403,9.36%
образование,3643,370,9.22%
свадьба,2138,186,8.00%
недвижимость,10029,782,7.23%


**Вывод**

По полученным данным отчетливо видно, что больше всего займов приходится на операции с недвижимостью, а также, что процент должников по данной категории заемщиков самый низкий. Это не удивительно, так как к своему жилью люди относятся гораздо трепетнее, ведь каждому хочется жить в комфортных условиях. Автомобиль, образование и свадьба являются для заемщиков менее приоритетными задачами.

## Шаг 4. Общий вывод

В ходе выполнения исследования по данным, предоставленным заказчиком:
1. Проведено **знакомство** с данными
2. Произведена **предобработка** данных:
- поиск и обработка пропущенных значений;
- замена типов данных с _float64_ на _int64_;
- поиск и обработка дубликатов;
- обработка артефактов и аномальных значений таблицы;
- лемматизация целей получения кредитного займа;
- созданы необходимые для ответов на поставленные вопросы категории

Также, в ходе ислледования зависимости возврата кредита в срок от категории заемщиков были получены **ответы на поставленные вопросы**:
1. Возврат кредита в срок действительно зависит от наличия детей в семье - бездетные семьи берут займы чаще семей с детьми (14167 займов против 7287 займов соответственно, разница почти в 2 раза) и относятся к обязательствам добросовестнее (7.56% просрочек против 8.16%, 9.17% и 9.45% соответственно, разница более 0.6%)
2. Имеется определенная зависимость между семейным положением заемщика и возвратом кредита в срок - люди, побывавшие в браке, добросовестнее относятся к своим обязательнствам (7.55%, 7.11% и 6.57% просрочек против 9.75% и 9.35% семей, никогда не состоящих в браке, разница около 2%)
3. Между уровнем дохода и возвратом кредита в срок имеется неожиданная зависимость - люди с малым доходом добросовестнее относятся к своим кредитным обязательствам, чем люди с более высоким (к примеру, заемщики с доходом до 36000 просрочили возврат займа в 5.34% случаев в отличии от заемщиков с доходом от 100000 до 150000, которые допустили просрочки в 8.72% случаев, разница целых 3.38%)
4. Цель получения кредита влияет на возврат кредита в срок, а именно, к кредитным обязательствам по операциям с жильем заемщики относятся добросовестнее (7.23% просрочек против 8%, 9.22% и 9.36% просрочек займов на иные цели)